# Reddit stock market prediction

### Import libraries

In [2]:
# pull libraries
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split

### Preprocess data

In [3]:
# pull data
djia_data = pd.read_csv('./stocknews/Combined_News_DJIA.csv', parse_dates=True)

djia_data['combined'] = djia_data[djia_data.columns[2:]].apply(
    lambda x: ' '.join(x.astype(str)),
    axis=1
)

def sanitise_row(row):
    return re.sub('[^A-Za-z ]+', '', row.replace("b\"", "").replace("b'", ""))

def remove_stop_words(sentence):
    sentence = sanitise_row(sentence)
    stop_words = set(stopwords.words('english')) 
    word_tokens = word_tokenize(sentence) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    return ' '.join(filtered_sentence)

djia_data['combined'] = djia_data['combined'].map(lambda x: remove_stop_words(x))

# print(type(djia_data['combined']))

In [4]:
# Ratio of label 1s and 0s
data_label = np.array(djia_data['Label'])

print(np.sum(data_label == 1) / data_label.shape[0])

0.5354449472096531


### Preprocess data

In [15]:
# prepare train and test data
all_data = djia_data['combined']
data_label = np.array(djia_data['Label'])

# vectorizer = CountVectorizer()
# X = vectorizer.fit_transform(all_data)
# print(vectorizer.get_feature_names())
# print(X.shape)

X_train, X_test, y_train, y_test = train_test_split(
    all_data, data_label, test_size=0.50, random_state=23)
# we also need dictionary that maps word to number of occurences -> 
# to handle cases where word is not found in the training corpus

In [63]:
# We need to convert training data into a vector for LSTM
# how does the vector look like?
# need to build word_index first!

def gen_word_indexes(data): 
    word_index = {}
    word_index["<PAD>"] = 0
    word_index["<START>"] = 1
    word_index["<UNK>"] = 2  # unknown
    word_index["<UNUSED>"] = 3

    index = 4
    # loop through everything in all_data
    for row in data:
        for word in row.split():
            word = word.lower()
            if word not in word_index:
                word_index[word] = index
                index += 1

    reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
    return (word_index, reverse_word_index)


def vectorise_text_data(data, word_index):
    vectorised_data = []
    for row in data:
        current_row = []
        for word in row.split():
            word = word.lower()
            current_row.append(word_index[word])
        vectorised_data.append(current_row)
    return vectorised_data

In [68]:
(word_index, reverse_word_index) = gen_word_indexes(all_data)
X_vectorised = vectorise_text_data(all_data, word_index)

In [121]:
from keras.preprocessing import sequence

# number of most-frequent words to use
nb_words = 40000
# cut texts after this number of words
maxlen = 80

X_v_train, X_v_test, y_v_train, y_v_test = train_test_split(
    X_vectorised, data_label, test_size=0.40, random_state=23)

X_v_pad_train = sequence.pad_sequences(X_v_train,
                                       value=word_index["<PAD>"],
                                       padding='post',
                                       maxlen=maxlen)
X_v_pad_test = sequence.pad_sequences(X_v_test, 
                                      value=word_index["<PAD>"],
                                      padding='post',
                                      maxlen=maxlen)

print(X_v_pad_train.shape, X_v_pad_test.shape)
print(X_v_pad_train[2])

(1193, 80) (796, 80)
[ 2512 16506 12074 16654 16655 16656 16657  2253   195 12074  1047  4551
  4713  6332 16658  1047  4996 10159    27    71  2215  4613  1047  8849
  1344   285  6204 11106  6792 16161  1193  2496 16644 16369 16659  1616
   571  2012 16660  4875   712   713  6624   802  4916   617  1406  3799
 16661   723   310 16662  3071  2206  3389  6107 13766   672  5422 16663
 16664  2419  7464  2413  1101   600 11961    89 12168    83   520   156
  2079  1057 16665    19  1340  1117   932 16666]


Now what? We train!

### Training using Naive Bayes

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

# LogisticRegression(random_state=0, solver='lbfgs')

v = CountVectorizer()

text_clf = Pipeline([
    ('vect', v),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB())
])

text_clf.fit(X_train, y_train)

predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)  

/Users/muhammad/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


0.5386934673366834

### Training using LSTM

In [122]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, SpatialDropout1D
from keras.layers import LSTM

EMBEDDING_DIM = 128 # dimension for dense embeddings for each token
LSTM_DIM = 64 # total LSTM units

model = Sequential()
model.add(Embedding(input_dim=nb_words, output_dim=EMBEDDING_DIM, input_length=maxlen))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(LSTM_DIM, dropout=0.8, recurrent_dropout=0.7))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss="binary_crossentropy", optimizer="adam",
              metrics=["accuracy"])

In [ ]:
batch_size = 128
model.fit(X_v_pad_train, y_v_train, epochs=5, batch_size=batch_size, 
          shuffle=True, validation_split=0.25, verbose=1)

Train on 894 samples, validate on 299 samples
Epoch 1/5
894/894 [==============================] - 2s 2ms/step - loss: 0.0924 - acc: 0.9911 - val_loss: 0.8865 - val_acc: 0.5351
Epoch 2/5
894/894 [==============================] - 2s 2ms/step - loss: 0.0832 - acc: 0.9855 - val_loss: 0.9357 - val_acc: 0.5418
Epoch 3/5
640/894 [====================>.........] - ETA: 0s - loss: 0.0759 - acc: 0.9891

In [132]:
scores = model.evaluate( X_v_pad_test, y_v_test, verbose=2)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 53.02%
